# 残差网络（ResNet）

As we design increasingly deeper networks it becomes imperative to understand how adding layers can increase the complexity and expressiveness of the network.

## 函数类

* Consider $\mathcal{F}$, the class of functions that a specific network architecture (together with learning rates and other hyperparameter settings) can reach.
* 现在假设$f^*$是我们真正想要找到的函数
    * 如果是$f^* \in \mathcal{F}$，那我们可以轻而易举的训练得到它
    * 但通常我们不会那么幸运
    
      \begin{equation*}
      f^*_\mathcal{F} := \mathop{\mathrm{argmin}}_f L(\mathbf{X}, \mathbf{y}, f) \text{ subject to } f \in \mathcal{F}
      \end{equation*}

## 函数类

* 怎样得到更近似真正$f^*$的函数呢
* 如果$\mathcal{F} \not\subseteq \mathcal{F}'$，则无法保证新的体系“更近似”

<center><img src="../img/functionclasses.svg" width="70%"></center>
<center>对于非嵌套函数类，较复杂（由较大区域表示）的函数类不能保证更接近“真”函数（ $f^*$ ）</center>

## 函数类

* 只有当较复杂的函数类包含较小的函数类时，我们才能确保提高它们的性能
* 对于深度神经网络，如果我们能将新添加的层训练成*恒等映射*（identity function）$f(\mathbf{x}) = \mathbf{x}$，新模型和原模型将同样有效

## 函数类
* 针对这一问题，何恺明等人提出了*残差网络*（ResNet）
* 它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计
* 残差网络的核心思想是：每个附加层都应该更容易地包含原始函数作为其元素之一

## 残差块

* 只需将右图虚线框内上方的加权运算（如仿射）的权重和偏置参数设成0，那么$f(\mathbf{x})$即为恒等映射

<center><img src="../img/residual-block.svg" width="30%"></center>
<center>一个正常块（左图）和一个残差块（右图）</center>

In [1]:
import torch
from d2l import torch as d2l
from torch import nn
from torch.nn import functional as F


class Residual(nn.Module):  # @save
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(
            input_channels, num_channels, kernel_size=3, padding=1, stride=strides
        )
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(
                input_channels, num_channels, kernel_size=1, stride=strides
            )
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

ModuleNotFoundError: No module named 'd2l'

## 残差块

* 当`use_1x1conv=True`时，添加通过$1 \times 1$卷积调整通道和分辨率

<center><img src="../img/resnet-block.svg" width="50%"></center>
<center>包含以及不包含 $1 \times 1$ 卷积层的残差块</center>

输入和输出形状一致

In [ ]:
blk = Residual(3, 3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

增加输出通道数的同时，减半输出的高和宽

In [ ]:
blk = Residual(3, 6, use_1x1conv=True, strides=2)
blk(X).shape

## ResNet模型

<center><img src="../img/resnet18.svg" width="20%"></center>
<center>ResNet-18 架构</center>

## ResNet模型

* ResNet的前两层跟之前介绍的GoogLeNet中的一样：
在输出通道数为64、步幅为2的$7 \times 7$卷积层后，接步幅为2的$3 \times 3$的最大池化层
* 不同之处在于ResNet每个卷积层后增加了批量归一化层

In [ ]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

## ResNet模型

* GoogLeNet在后面接了4个由Inception块组成的模块
* ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块
* 第一个模块的通道数同输入通道数一致
* 之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半
* 阅读：残差网络\_ResNet.pdf

In [ ]:
def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(
                Residual(input_channels, num_channels, use_1x1conv=True, strides=2)
            )
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [ ]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

## ResNet模型

* 最后，与GoogLeNet一样，在ResNet中加入全局平均池化层，以及全连接层输出

In [ ]:
net = nn.Sequential(
    b1, b2, b3, b4, b5, nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(512, 10)
)

## ResNet模型

* 每个模块有4个卷积层（不包括恒等映射的$1\times 1$卷积层）
* 加上第一个$7\times 7$卷积层和最后一个全连接层，共有18层
* 因此，这种模型通常被称为ResNet-18

In [ ]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "output shape:\t", X.shape)

## 总结

* 残差块使得很深的网络更加容易训练
    * 甚至可以训练一千层的网络
* 残差网络对随后的深层神经网络设计产生了深远影响，无论是卷积类网络还是全连接类网络